# Spectrum Query Tuning

In this lab, we show you how to diagnose your Redshift Spectrum query performance and optimize performance by leveraging partitions, optimizing storage, and predicate pushdown.

## Querying with Amazon Redshift Spectrum

Create a star schema data model by creating dimension tables in your Redshift cluster, and fact tables in S3 as show in the diagram below.

1. Create the dimension tables by running this script from your client tool.
    ```sql
    DROP TABLE IF EXISTS customer;
    CREATE TABLE customer (
    c_custkey     	integer        not null sortkey,
    c_name        	varchar(25)    not null,
    c_address     	varchar(25)    not null,
    c_city        	varchar(10)    not null,
    c_nation      	varchar(15)    not null,
    c_region      	varchar(12)    not null,
    c_phone       	varchar(15)    not null,
    c_mktsegment      varchar(10)    not null)
    diststyle all;

    DROP TABLE IF EXISTS dwdate;
    CREATE TABLE dwdate (
    d_datekey            integer       not null sortkey,
    d_date               varchar(19)   not null,
    d_dayofweek	      varchar(10)   not null,
    d_month      	    varchar(10)   not null,
    d_year               integer       not null,
    d_yearmonthnum       integer  	 not null,
    d_yearmonth          varchar(8)	not null,
    d_daynuminweek       integer       not null,
    d_daynuminmonth      integer       not null,
    d_daynuminyear       integer       not null,
    d_monthnuminyear     integer       not null,
    d_weeknuminyear      integer       not null,
    d_sellingseason      varchar(13)    not null,
    d_lastdayinweekfl    varchar(1)    not null,
    d_lastdayinmonthfl   varchar(1)    not null,
    d_holidayfl          varchar(1)    not null,
    d_weekdayfl          varchar(1)    not null)
    diststyle all;
    ```
1. Load data into your dimension table by running the following script. You will need to provide an IAM role with the permissions to run the COPY command on your cluster. You can use the IAM role that you identified earlier. This will load the data set from S3 into your Redshift cluster. Expect the script to take a few minutes to complete. The customer and time dimension consists of 3M records, and 2556 records respectively.
    ```sql
    copy customer from 's3://awssampledbuswest2/ssbgz/customer'
    iam_role default
    gzip region 'us-west-2';

    copy dwdate from 's3://awssampledbuswest2/ssbgz/dwdate'
    iam_role default
    gzip region 'us-west-2';
    ```
1. Next, create an External Schema that references datasets that reside outside of your Redshift cluster. Define this schema by running the following command. You will need to provide an IAM role with the permissions to read S3 date from your cluster. This should be the same role used above in the COPY command. Redshift stores the meta-data that describes your external databases and schemas in the AWS Glue data catalog by default. Once created, you can view the schema from Glue or Athena.
    ```sql
    CREATE EXTERNAL SCHEMA clickstream
    from data catalog database 'clickstream'
    iam_role default
    CREATE EXTERNAL DATABASE IF NOT EXISTS;
    ```
1. Use the AWS Glue Crawler to create your external table clickstream.clickstream-csv10 and clickstream.clickstream-parquet1 under locations s3://wysde-datasets/clickstream.
    - Navigate to the Glue Crawler Page
    - Add Crawler, and enter the crawler name clickstream and click Next
    - Choose S3 as the data store and the include path of s3://wysde-datasets/clickstream
    - Click on Add database. Name it clickstream and click Create. Select the Database clickstream from the list
    - Select all remaining defaults. Once the Crawler has been created, click on Run Crawler.
    - Once the Crawler has completed its run, you will see two new tables in the Glue Catalog.
    - For uservisits_csv10 table, we need to correct the schema. Click on Edit Schema and adjust the column names and datatypes as per the table below. Click Save.
    ```
    | #  | Column name    | Data type | Partition key | Comment |
    | -- | -------------- | --------- | ------------- | ------- |
    | 1  | adrevenue      | double    | \-            | \-      |
    | 2  | countrycode    | string    | \-            | \-      |
    | 3  | custkey        | bigint    | \-            | \-      |
    | 4  | desturl        | string    | \-            | \-      |
    | 5  | duration       | bigint    | \-            | \-      |
    | 6  | languagecode   | string    | \-            | \-      |
    | 7  | searchword     | string    | \-            | \-      |
    | 8  | sourceip       | string    | \-            | \-      |
    | 9  | useragent      | string    | \-            | \-      |
    | 10 | visitdate      | bigint    | \-            | \-      |
    | 11 | yearmonthkey   | bigint    | \-            | \-      |
    | 12 | customer       | string    | Partition (0) | \-      |
    | 13 | visityearmonth | string    | Partition (1) | \-      |
    ```
1. Navigate back to your SQL Client tool and run the query below. This query performs a join between dimension tables in Redshift, and the clickstream fact table in S3 effectively blending data from the data Lake and data warehouse. The ad revenue data originates from S3 while the customer and time attributes like market segment originate from the dimension tables in Redshift.
    ```sql
    SELECT c.c_name, c.c_mktsegment, t.prettyMonthYear, SUM(uv.adRevenue)
    FROM clickstream.uservisits_csv10 as uv
    RIGHT OUTER JOIN customer as c ON c.c_custkey = uv.custKey
    INNER JOIN (
    SELECT DISTINCT d_yearmonthnum, (d_month||','||d_year) as prettyMonthYear
    FROM dwdate
    WHERE d_yearmonthnum >= 199410) as t ON uv.yearMonthKey = t.d_yearmonthnum
    WHERE c.c_custkey <= 5
    GROUP BY c.c_name, c.c_mktsegment, t.prettyMonthYear, uv.yearMonthKey
    ORDER BY c.c_name, c.c_mktsegment, uv.yearMonthKey  ASC
    ```

Expect this query to take a few minutes to complete as nearly 3.8 billion records will be accessed. The results of the query should be as follows: 

![](https://static.us-east-1.prod.workshops.aws/public/0c9aa4a3-ef98-4fc8-8579-9f6cd75acdae/static/images/spectrum_query_results_01.png)

## Performance Diagnostics

There are a few utilities that provide visibility into Redshift Spectrum:

- EXPLAIN  - Provides the query execution plan, which includes info around what processing is pushed down to Spectrum. Steps in the plan that include the prefix S3 are executed on Spectrum; for instance, the plan for the query above has a step “S3 Seq Scan clickstream.uservisits_csv10” indicating that Spectrum performs a scan on S3 as part of the query execution.
- SVL_S3QUERY_SUMMARY  - Provides statistics for Redshift Spectrum queries are stored in this table. While the execution plan presents cost estimates, this table stores actual statistics of past query runs.
- SVL_S3PARTITION  - Provides details about Amazon Redshift Spectrum partition pruning at the segment and node slice level.

The diagnostics reveal why our query took so long.

Run the following query on the SYS_EXTERNAL_QUERY_DETAIL table(Serverless Cluster):

```sql
SELECT
 query_id,
 file_location,
 End_time - Start_time as elapsed,
 total_partitions,
 qualified_partitions,
 scanned_files,
 returned_rows,
 returned_bytes
FROM SYS_EXTERNAL_QUERY_DETAIL
ORDER  BY query_id, start_time DESC;
```

![](https://static.us-east-1.prod.workshops.aws/public/0c9aa4a3-ef98-4fc8-8579-9f6cd75acdae/static/images/lab5_spectrum_new_1.png)

Run the same Redshift Spectrum query again, but with EXPLAIN

```sql
EXPLAIN
SELECT c.c_name, c.c_mktsegment, t.prettyMonthYear, SUM(uv.adRevenue)
FROM clickstream.uservisits_csv10 as uv
RIGHT OUTER JOIN customer as c ON c.c_custkey = uv.custKey
INNER JOIN (
  SELECT DISTINCT d_yearmonthnum, (d_month||','||d_year) as prettyMonthYear
  FROM dwdate WHERE d_yearmonthnum >= 199810) as t ON uv.yearMonthKey = t.d_yearmonthnum
WHERE c.c_custkey <= 3
GROUP BY c.c_name, c.c_mktsegment, t.prettyMonthYear, uv.yearMonthKey
ORDER BY c.c_name, c.c_mktsegment, uv.yearMonthKey  ASC
```

The output will look similar to the example below. Don’t worry about understanding the details of the query plan at this time.
The takeaway is that the query plan reveals how Redshift Spectrum is leveraged in the query. The highlighted line below indicates that Redshift Spectrum is leveraged as part of the query execution to perform a scan. It also reveals that partitions were not used.

![](https://static.us-east-1.prod.workshops.aws/public/0c9aa4a3-ef98-4fc8-8579-9f6cd75acdae/static/images/lab5_spectrum_2.png)

## Optimizing with Partitions

In this section, you will learn about partitions, and how they can be used to improve the performance of your Redshift Spectrum queries. Partitioning is a key means to improving scan efficiency. Previously, we ran the glue crawler which created our external tables along with partitions. Navigate back to the Glue Catalog https://console.aws.amazon.com/glue/home?#catalog:tab=tables . Click on the uservisits_csv10 table. The columns customer and visityearmonth are set as the partition keys.

If you have interest in understanding the details of how partitions were setup, refer to the documentation. You may explore the S3 buckets that are serving our Redshift Spectrum datasets by navigating to the following location: https://s3.console.aws.amazon.com/s3/buckets/redshift-immersionday-labs/data/clickstream/uservisits_csv10/ 

The entire 3.8 billion-row dataset is organized as a collection of large files where each file contains data exclusive to a particular customer and month in a year. This allows you to partition your data into logical subsets by customer and year/month as exemplified above. With partitions, the query engine can target a subset of files:

- Only for specific customers
- Only data for specific months
- A combination of specific customers and year/months

Take note that the right choice of partitions is dependent on your workload. Partitions should be selected based on the primary queries you want to optimize, and your data profile. For those implementing their own clickstream analytics, a partition scheme like year/month/region often makes sense. The choice of using customer in the partition scheme isn’t optimal for a use case where there is a very large number of customers, and little data for each one. The data set, and scheme used in this example is a practical one for scenarios like a multi-tenant ad-tech platform, or an IoT platform. In these cases, there are a moderate number of customers (tenants), and a lot of data per customer.

Observe the effects of leveraging partitioning on our query by running the following query.

```sql
SELECT c.c_name, c.c_mktsegment, t.prettyMonthYear, SUM(uv.adRevenue)
FROM clickstream.uservisits_csv10 as uv
RIGHT OUTER JOIN customer as c ON c.c_custkey = uv.customer
INNER JOIN
  (SELECT DISTINCT d_yearmonthnum, (d_month||','||d_year) as prettyMonthYear
   FROM dwdate
   WHERE d_yearmonthnum >= 199810) as t ON uv.yearMonthKey = t.d_yearmonthnum
WHERE c.c_custkey <= 3
GROUP BY c.c_name, c.c_mktsegment, t.prettyMonthYear, uv.yearMonthKey
ORDER BY c.c_name, c.c_mktsegment, uv.yearMonthKey  ASC
```

The join condition from the previous query has been modified. Instead of joining on the synthetic key, custKey, we use the partition key, customer, that we created as part of the data modeling process. This query should run approximately 2X faster than the previous.

Run the same Redshift Spectrum query again, but with EXPLAIN. Unlike before, you should see a Filter clause as part of the PartitionInfo scan that indicates partition pruning is executed as part of the query plan:

```
->  XN Seq Scan PartitionInfo of clickstream.uservisits_csv10 uv  (cost=0.00..12.50 rows=334 width=4)
         Filter: ((customer <= 3) AND (subplan 4: (customer = $2)))
```

Re-run the system query:

```sql
SELECT
 query_id,
 file_location,
 End_time - Start_time as elapsed,
 total_partitions,
 qualified_partitions,
 scanned_files,
 returned_rows,
 returned_bytes
FROM SYS_EXTERNAL_QUERY_DETAIL
ORDER  BY query_id, start_time DESC;
```

You should observe the following results in Serverless Cluster: 

![](https://static.us-east-1.prod.workshops.aws/public/0c9aa4a3-ef98-4fc8-8579-9f6cd75acdae/static/images/lab5_spectrum_30.png)

Note that s3_scanned_rows reveals that the rows scanned has been halved when compared with the previous query. This explains why our query ran roughly twice as fast.

The results are due to the fact that our data is evenly distributed across all customers, and by querying 3 of 6 customers with our customer partition key, the database engine is able to intelligently scan the subset of data containing customers 1,2 and 3 instead of the entire data set. However, the scan is still very inefficient, and we can benefit from utilizing our year/month partition key as well.

Run the query below:

```sql
SELECT c.c_name, c.c_mktsegment, t.prettyMonthYear, SUM(uv.adRevenue)
FROM clickstream.uservisits_csv10 as uv
RIGHT OUTER JOIN customer as c ON c.c_custkey = uv.customer
INNER JOIN (
  SELECT DISTINCT d_yearmonthnum, (d_month||','||d_year) as prettyMonthYear
  FROM dwdate
  WHERE d_yearmonthnum >= 199810) as t ON uv.visitYearMonth = t.d_yearmonthnum -- << Change >> ON uv.yearMonthKey = t.d_yearmonthnum
WHERE c.c_custkey <= 3
GROUP BY c.c_name, c.c_mktsegment, t.prettyMonthYear, uv.yearMonthKey
ORDER BY c.c_name, c.c_mktsegment, uv.yearMonthKey  ASC
```

The join condition from the previous query has been modified. Instead of joining on the synthetic key, yearMonthKey, we use the partition key, visitYearMonth. Our latest query utilizes both customer and time partitions and if you run this query a few times, you should see execution time in the range of 8s, which is a 22.5X improvement on our original query!

Re-run the System query:

```sql
SELECT
 query_id,
 file_location,
 End_time - Start_time as elapsed,
 total_partitions,
 qualified_partitions,
 scanned_files,
 returned_rows,
 returned_bytes
FROM SYS_EXTERNAL_QUERY_DETAIL
ORDER  BY query_id, start_time DESC;
```

Upon reviewing the statistics for this query, you should observe that Redshift Spectrum scans and returns the exact number of rows (66,270,117) required to compute the query.

You should observe the following results in Serverless Cluster: 

![](https://static.us-east-1.prod.workshops.aws/public/0c9aa4a3-ef98-4fc8-8579-9f6cd75acdae/static/images/lab5_spectrum_40.png)

## Predicate Pushdown

In the last section, we learned that Spectrum excels at performing large aggregations. In this section, we’ll experiment the results of pushing more work down to Redshift Spectrum.

Run the following query. After running this query a few times, you should observe execution times in the range of 4 seconds.

```sql
SELECT c.c_name, c.c_mktsegment, t.prettyMonthYear, uv.totalRevenue
FROM (
  SELECT customer, visitYearMonth, SUM(adRevenue) as totalRevenue
  FROM clickstream.uservisits_parquet1
  WHERE customer <= 3 and visitYearMonth >= 199810
  GROUP BY  customer, visitYearMonth) as uv
RIGHT OUTER JOIN customer as c ON c.c_custkey = uv.customer
INNER JOIN (
  SELECT DISTINCT d_yearmonthnum, (d_month||','||d_year) as prettyMonthYear
  FROM dwdate WHERE d_yearmonthnum >= 199810) as t ON uv.visitYearMonth = t.d_yearmonthnum
ORDER BY c.c_name, c.c_mktsegment, uv.visitYearMonth ASC;
```

This query improves on our previous one in a couple of ways.

- We are querying the clickstream.uservisits_parquet1 table instead of clickstream.uservisits_csv10. These two tables contain the same data set, but they have been processed in different ways. The table clickstream.uservisits_parquet1 contains data in parquet format. Parquet is a columnar format, and yields I/O benefits for analytical workloads by providing compression and efficient retrieval of the attributes that are selected by the queries. Furthermore, the “1” vs “10” suffix indicates that all the data for each partition is stored in a single file instead of ten files like the CSV data set. The latter case has less overhead involved in processing large scans and aggregations.
- The aggregation work has been pushed down to Redshift Spectrum. When we analyzed the query plan previously, we observed that Spectrum is used for scanning. When you analyze the above query, you will see that aggregations are also performed at the Spectrum layer.

Re-run the System query:

```sql
SELECT
 query_id,
 file_location,
 End_time - Start_time as elapsed,
 total_partitions,
 qualified_partitions,
 scanned_files,
 returned_rows,
 returned_bytes
FROM SYS_EXTERNAL_QUERY_DETAIL
ORDER  BY query_id, start_time DESC;
```

You should observe the following results in Serverless Cluster: 

![](https://static.us-east-1.prod.workshops.aws/public/0c9aa4a3-ef98-4fc8-8579-9f6cd75acdae/static/images/lab5_spectrum_50.png)

The statistics reveal the source of some of the performance improvements:

- The bytes scanned is reduced even though the same number of rows are scanned as a result of compression.
- The number of rows returned/scanned files is reduced to 9 from ~66.3M. This is the result of pushing the aggregation down to the Spectrum layer. Our data is stored at the day-level granularity, and our query rolls that up to the month-level. By pushing the aggregation down to the Spectrum fleet, we only need to return 9 records that aggregate ad revenue up to the month level so that they can be joined with the required dimension attributes.

Run the query again with EXPLAIN:

The query plan should include an S3 Aggregate step, which indicates that the Spectrum layer offloads the aggregation processing for this query.

```
QUERY PLAN
XN Merge  (cost=1000094008880.16..1000094008880.18 rows=7 width=78)
  Merge Key: c.c_name, c.c_mktsegment, uv.visityearmonth
  ->  XN Network  (cost=1000094008880.16..1000094008880.18 rows=7 width=78)
        Send to leader
        ->  XN Sort  (cost=1000094008880.16..1000094008880.18 rows=7 width=78)
              Sort Key: c.c_name, c.c_mktsegment, uv.visityearmonth
              ->  XN Hash Join DS_DIST_ALL_NONE  (cost=94008878.97..94008880.06 rows=7 width=78)
                    Hash Cond: ("outer".customer = "inner".c_custkey)
                    ->  XN Hash Join DS_DIST_ALL_NONE  (cost=93971378.97..93971379.61 rows=7 width=48)
                          Hash Cond: ("outer".visityearmonth = "inner".d_yearmonthnum)
                          ->  XN Subquery Scan uv  (cost=93971346.13..93971346.42 rows=23 width=16)
                                ->  XN HashAggregate  (cost=93971346.13..93971346.19 rows=23 width=16)
                                      ->  XN Partition Loop  (cost=93969358.63..93970506.13 rows=112000 width=16)
                                            ->  XN Seq Scan PartitionInfo of clickstream.uservisits_parquet1  (cost=0.00..17.50 rows=112 width=8)
                                                  Filter: ((customer <= 3) AND (visityearmonth >= 199810) AND (visityearmonth >= 199810))
                                            ->  XN S3 Query Scan uservisits_parquet1  (cost=46984679.32..46984689.32 rows=1000 width=8)
                                                  ->  S3 Aggregate  (cost=46984679.32..46984679.32 rows=1000 width=8)
                                                        ->  S3 Seq Scan clickstream.uservisits_parquet1 location:"s3://redshift-spectrum-datastore-parquet1" format:PARQUET  (cost=0.00..37587743.45 rows=3758774345 width=8)
                          ->  XN Hash  (cost=32.82..32.82 rows=7 width=36)
                                ->  XN Subquery Scan t  (cost=0.00..32.82 rows=7 width=36)
                                      ->  XN Unique  (cost=0.00..32.75 rows=7 width=18)
                                            ->  XN Seq Scan on dwdate  (cost=0.00..32.43 rows=64 width=18)
                                                  Filter: (d_yearmonthnum >= 199810)
                    ->  XN Hash  (cost=30000.00..30000.00 rows=3000000 width=38)
                          ->  XN Seq Scan on customer c  (cost=0.00..30000.00 rows=3000000 width=38)
```

## Native Redshift versus Redshift with Spectrum

At this point, you might be asking yourself, why would I ever not use Spectrum? Well, you still get additional value from loading data into Redshift. In fact, it turns out that our last query runs even faster when executed exclusively in native Redshift. Running a full test is beyond the time we have for the lab, so let’s review test results that compares running the last query with Redshift Spectrum versus exclusively with Redshift on various cluster sizes.

As a rule of thumb, queries that aren’t dominated by I/O and involve multiple joins are better optimized in native Redshift. Furthermore, the variability in latency in native Redshift is significantly lower. For use cases where you have tight performance SLAs on queries, you may want to consider using Redshift exclusively to support those queries.

On the other hand, when you have the need to perform large scans, you could benefit from the best of both worlds: higher performance at lower cost. For instance, imagine we needed to enable our business analysts to interactively discover insights across a vast amount of historical data.
